# Instalar librerías necesarias

In [ ]:
!pip install google-generativeai scikit-learn
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.6 MB/s eta 0

In [ ]:
# Importar librerías
import pandas as pd
import google.generativeai as genai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from google.colab import drive
import os

# Leemos discursos

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_discursos = '/content/drive/MyDrive/Clases/NLP para ciencias sociales/99. Datos'

def buscar_discursos_en_ruta(ruta):
    """Buscar archivos .txt en una ruta específica"""
    archivos_encontrados = []

    if not os.path.exists(ruta):
        print(f"❌ La ruta {ruta} no existe")
        return []

    # Buscar archivos .txt
    for archivo in os.listdir(ruta):
        if archivo.endswith('.txt'):
            ruta_completa = os.path.join(ruta, archivo)
            archivos_encontrados.append(ruta_completa)

    return archivos_encontrados

In [ ]:
archivos_txt = buscar_discursos_en_ruta(ruta_discursos)

print(f"📄 Archivos encontrados: {len(archivos_txt)}")
if len(archivos_txt) > 0:
    for archivo in archivos_txt:
        nombre = os.path.basename(archivo)
        print(f"   - {nombre}")
else:
    print("❌ No se encontraron archivos .txt en la ruta especificada")
    print(f"💡 Verifica que la ruta sea correcta: {ruta_discursos}")
    print("💡 Asegúrate de que los archivos tengan extensión .txt")

📄 Archivos encontrados: 8
   - discursos_petro.txt
   - discursos_duque.txt
   - discurso_posesion_petro.txt
   - discurso_posesion_duque.txt
   - discurso_onu_petro.txt
   - discurso_onu_duque.txt
   - discursos_posesion_uribe.txt
   - discursos_posesion_santos.txt


### procesamos los discursos

In [ ]:
documentos = []
metadatos = []

# Función para extraer político del nombre del archivo
def extraer_politico(nombre_archivo):
    nombre = nombre_archivo.lower()
    if 'petro' in nombre:
        return 'Petro'
    elif 'duque' in nombre:
        return 'Duque'
    elif 'uribe' in nombre:
        return 'Uribe'
    elif 'santos' in nombre:
        return 'Santos'
    else:
        return 'Político'

In [ ]:
# Procesar archivos reales
for archivo in archivos_txt:
    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()

        nombre_archivo = os.path.basename(archivo)
        politico = extraer_politico(nombre_archivo)

        # Dividir en párrafos
        parrafos = contenido.split('\n\n')
        fragmentos_validos = 0

        for i, parrafo in enumerate(parrafos):
            if len(parrafo.strip()) > 100:  # Solo párrafos con contenido suficiente
                documentos.append(parrafo.strip())
                metadatos.append({
                    'politico': politico,
                    'archivo': nombre_archivo,
                    'fragmento': i+1
                })
                fragmentos_validos += 1

        print(f"✅ {nombre_archivo}: {politico} - {fragmentos_validos} fragmentos")

    except Exception as e:
        print(f"❌ Error leyendo {archivo}: {e}")

✅ discursos_petro.txt: Petro - 1 fragmentos
✅ discursos_duque.txt: Duque - 1 fragmentos
✅ discurso_posesion_petro.txt: Petro - 1 fragmentos
✅ discurso_posesion_duque.txt: Duque - 1 fragmentos
✅ discurso_onu_petro.txt: Petro - 1 fragmentos
✅ discurso_onu_duque.txt: Duque - 1 fragmentos
✅ discursos_posesion_uribe.txt: Uribe - 81 fragmentos
✅ discursos_posesion_santos.txt: Santos - 249 fragmentos


In [ ]:
# Verificar que se procesaron documentos
if len(documentos) == 0:
    print("❌ ERROR: No se pudo procesar ningún documento")
    print("💡 Verifica que los archivos .txt contengan texto válido")
    raise ValueError("No se procesaron documentos válidos")

print(f"\n📊 TOTAL: {len(documentos)} fragmentos procesados")


📊 TOTAL: 336 fragmentos procesados


# cargamos el modelo

In [ ]:
from google.colab import userdata
import os
google_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# Configurar API (necesitas tu API key)
genai.configure(api_key=google_api_key)

# Embeddgins y vectorstore

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

In [ ]:
modelo_embeddings = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
cliente = chromadb.Client()
coleccion = cliente.create_collection(name="discursos_politicos")

In [ ]:
# Crear IDs únicos para cada documento
ids = []
for i in range(len(documentos)):
    ids.append(f"doc_{i}")

# Preparar metadatos
metadatos_chroma = []
for meta in metadatos:
    metadatos_chroma.append({
        "politico": meta["politico"],
        "archivo": meta["archivo"]
    })

# Agregar todo a ChromaDB (calcula embeddings automáticamente)
coleccion.add(
    documents=documentos,
    metadatas=metadatos_chroma,
    ids=ids
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 58.1MiB/s]


# Probar búsqueda semántica

In [ ]:
pregunta_test = "¿Qué dice sobre drogas y narcotráfico?"
print(f"Pregunta: {pregunta_test}")

# Buscar documentos relevantes
resultados = coleccion.query(
    query_texts=[pregunta_test],
    n_results=3
)

print("\nDocumentos encontrados:")
for i in range(len(resultados["documents"][0])):
    doc_texto = resultados["documents"][0][i]
    doc_meta = resultados["metadatas"][0][i]
    distancia = resultados["distances"][0][i]
    similitud = 1 - distancia  # Convertir distancia a similitud

    print(f"\n{i+1}. {doc_meta['politico']} (similitud: {similitud:.3f})")
    print(f"   Texto: {doc_texto[:150]}...")


Pregunta: ¿Qué dice sobre drogas y narcotráfico?

Documentos encontrados:

1. Santos (similitud: 0.309)
   Texto: Estos jóvenes colombianos nos están dando un mensaje claro, nos están enseñando el camino de la grandeza y de la disciplina. ¡Y vamos a seguirlo!...

2. Santos (similitud: 0.277)
   Texto: Yo aspiro, durante mi gobierno, a sembrar las bases de una verdadera reconciliación entre los colombianos....

3. Santos (similitud: 0.275)
   Texto: En medio de la voluntad y el tesón de más de 45 millones de buenos colombianos, subsiste, sin embargo, una ínfima minoría que persiste, con el terrori...


# RAG Completo

In [ ]:
model = genai.GenerativeModel('gemini-2.0-flash-exp')

In [ ]:
pregunta = "¿Qué diferencias hay entre Petro y Duque sobre medio ambiente?"
print(f"Pregunta: {pregunta}")

# Buscar documentos
resultados = coleccion.query(
    query_texts=[pregunta],
    n_results=3
)

# Crear contexto con los documentos encontrados
contexto = ""
for i in range(len(resultados["documents"][0])):
    doc_texto = resultados["documents"][0][i]
    doc_meta = resultados["metadatas"][0][i]
    politico = doc_meta['politico']

    contexto += f"\nFUENTE {i+1} - {politico}:\n{doc_texto}\n"

Pregunta: ¿Qué diferencias hay entre Petro y Duque sobre medio ambiente?


In [ ]:
# Crear prompt para Gemini
prompt = f"""
Basándote en estas fuentes de discursos políticos, responde la pregunta:

{contexto}

PREGUNTA: {pregunta}

INSTRUCCIONES:
- Usa solo información de las fuentes
- Menciona qué político dijo qué cosa
- Si hay diferencias, explícalas claramente
- Sé objetivo y preciso

RESPUESTA:
"""

In [ ]:
# Generar respuesta
response = model.generate_content(prompt)
print(f"\n💬 RESPUESTA:")
print(response.text)


💬 RESPUESTA:
Las fuentes proporcionadas solo contienen discursos de Santos. Por lo tanto, no es posible identificar diferencias entre Petro y Duque sobre el medio ambiente basándose en esta información. Santos menciona en la FUENTE 1 que "trabajaremos por el medio ambiente, PARA PAGAR ESA DEUDA IMPOSTERGABLE A LAS NUEVAS GENERACIONES". No hay información sobre Petro o Duque en las fuentes.



## Diagnostico

In [ ]:
# Contar documentos por político
contador_politicos = {}
for meta in metadatos:
    politico = meta['politico']
    if politico in contador_politicos:
        contador_politicos[politico] += 1
    else:
        contador_politicos[politico] = 1

print("📊 Distribución de fragmentos por político:")
for politico, cantidad in contador_politicos.items():
    print(f"   {politico}: {cantidad} fragmentos")


📊 Distribución de fragmentos por político:
   Petro: 3 fragmentos
   Duque: 3 fragmentos
   Uribe: 81 fragmentos
   Santos: 249 fragmentos


In [ ]:
# Ver algunos ejemplos de cada político
print(f"\n📝 EJEMPLOS DE CONTENIDO:")
politicos_mostrados = set()

for i, (doc, meta) in enumerate(zip(documentos, metadatos)):
    politico = meta['politico']

    if politico not in politicos_mostrados and len(politicos_mostrados) < 4:
        print(f"\n--- {politico} ---")
        print(f"Archivo: {meta['archivo']}")
        print(f"Texto: {doc[:200]}...")
        politicos_mostrados.add(politico)


📝 EJEMPLOS DE CONTENIDO:

--- Petro ---
Archivo: discursos_petro.txt
Texto: Llegar aquí indudablemente implica recorrer una vida. La vida inmensa que nunca se recorre sola. Aquí está mi madre, Clara, nada existiría en mi mente en este momento sin ella. Aquí está mi padre, Gus...

--- Duque ---
Archivo: discursos_duque.txt
Texto: Nos encontramos de nuevo en este foro global que ha sido históricamente un espacio vital para el desarrollo del multilateralismo, la construcción global de la paz y de soluciones a las amenazas de nue...

--- Uribe ---
Archivo: discursos_posesion_uribe.txt
Texto: Retomemos el lazo unificador de la ley, la autoridad democrática, la libertad y la justicia social.
No venimos a quejarnos, llegamos a trabajar.
Las vidas de dos seres, disímiles en talante, formación...

--- Santos ---
Archivo: discursos_posesion_santos.txt
Texto: Nuestro país es una maravillosa combinación de culturas, de razas, de talentos, de riquezas naturales, que nos hace únicos en el planeta..

Santos tiene 249 fragmentos vs 3 de Petro y Duque. Esto hace que Santos domine las búsquedas. Ejecuta la parte de rebalanceado del código que está más abajo:

## Balanceamos datos

In [ ]:
# Limpiar y recrear documentos
ruta_discursos = '/content/drive/MyDrive/Clases/NLP para ciencias sociales/99. Datos'

# Buscar archivos nuevamente
archivos_txt = []
for archivo in os.listdir(ruta_discursos):
    if archivo.endswith('.txt'):
        ruta_completa = os.path.join(ruta_discursos, archivo)
        archivos_txt.append(ruta_completa)

print(f"📁 Procesando archivos desde: {ruta_discursos}")

# Limpiar y recrear documentos
documentos_nuevos = []
metadatos_nuevos = []

for archivo in archivos_txt:
    with open(archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()

    nombre_archivo = os.path.basename(archivo)

    # Extraer político
    if 'petro' in nombre_archivo.lower():
        politico = 'Petro'
    elif 'duque' in nombre_archivo.lower():
        politico = 'Duque'
    elif 'uribe' in nombre_archivo.lower():
        politico = 'Uribe'
    elif 'santos' in nombre_archivo.lower():
        politico = 'Santos'
    else:
        politico = 'Desconocido'

    # NUEVA FRAGMENTACIÓN: más inteligente
    # Dividir por párrafos y luego combinar hasta cierta longitud
    parrafos = contenido.split('\n\n')

    fragmento_actual = ""
    contador_fragmentos = 0

    for parrafo in parrafos:
        parrafo = parrafo.strip()

        # Saltar párrafos muy cortos
        if len(parrafo) < 30:
            continue

        # Si el fragmento actual + nuevo párrafo es muy largo, guardar el actual
        if len(fragmento_actual + " " + parrafo) > 800:
            if len(fragmento_actual) > 150:  # Solo guardar si tiene contenido suficiente
                documentos_nuevos.append(fragmento_actual.strip())
                metadatos_nuevos.append({
                    'politico': politico,
                    'archivo': nombre_archivo,
                    'fragmento': contador_fragmentos + 1
                })
                contador_fragmentos += 1
            fragmento_actual = parrafo
        else:
            if fragmento_actual:
                fragmento_actual += "\n\n" + parrafo
            else:
                fragmento_actual = parrafo

    # Agregar último fragmento
    if len(fragmento_actual.strip()) > 150:
        documentos_nuevos.append(fragmento_actual.strip())
        metadatos_nuevos.append({
            'politico': politico,
            'archivo': nombre_archivo,
            'fragmento': contador_fragmentos + 1
        })
        contador_fragmentos += 1

    print(f"✅ {nombre_archivo}: {politico} - {contador_fragmentos} fragmentos")


📁 Procesando archivos desde: /content/drive/MyDrive/Clases/NLP para ciencias sociales/99. Datos
✅ discursos_petro.txt: Petro - 1 fragmentos
✅ discursos_duque.txt: Duque - 1 fragmentos
✅ discurso_posesion_petro.txt: Petro - 1 fragmentos
✅ discurso_posesion_duque.txt: Duque - 1 fragmentos
✅ discurso_onu_petro.txt: Petro - 1 fragmentos
✅ discurso_onu_duque.txt: Duque - 1 fragmentos
✅ discursos_posesion_uribe.txt: Uribe - 34 fragmentos
✅ discursos_posesion_santos.txt: Santos - 71 fragmentos


In [ ]:
# Verificar nueva distribución
print(f"\n📊 NUEVA DISTRIBUCIÓN:")
contador_nuevos = {}
for meta in metadatos_nuevos:
    politico = meta['politico']
    if politico in contador_nuevos:
        contador_nuevos[politico] += 1
    else:
        contador_nuevos[politico] = 1

for politico, cantidad in contador_nuevos.items():
    print(f"   {politico}: {cantidad} fragmentos")


📊 NUEVA DISTRIBUCIÓN:
   Petro: 3 fragmentos
   Duque: 3 fragmentos
   Uribe: 34 fragmentos
   Santos: 71 fragmentos


Ante los problemas de balanceo, trabajaremos solo con Petro y Duque

In [ ]:
# Filtrar documentos existentes
documentos_filtrados = []
metadatos_filtrados = []

for i, meta in enumerate(metadatos):
    if meta['politico'] in ['Petro', 'Duque']:
        documentos_filtrados.append(documentos[i])
        metadatos_filtrados.append(meta)

print(f"Petro: {sum(1 for m in metadatos_filtrados if m['politico'] == 'Petro')} fragmentos")
print(f"Duque: {sum(1 for m in metadatos_filtrados if m['politico'] == 'Duque')} fragmentos")
print(f"Total: {len(documentos_filtrados)} fragmentos")

# Crear ChromaDB nuevo
try:
    cliente.delete_collection("discursos_politicos")
except:
    pass

coleccion = cliente.create_collection(name="petro_duque")

# Agregar documentos
ids = [f"doc_{i}" for i in range(len(documentos_filtrados))]
metadatos_chroma = [{"politico": m["politico"], "archivo": m["archivo"]} for m in metadatos_filtrados]

coleccion.add(
    documents=documentos_filtrados,
    metadatas=metadatos_chroma,
    ids=ids
)

print("✅ ChromaDB creado con solo Petro y Duque")

Petro: 3 fragmentos
Duque: 3 fragmentos
Total: 6 fragmentos
✅ ChromaDB creado con solo Petro y Duque


In [ ]:
# Probar búsqueda
pregunta = "¿Qué diferencias hay entre Petro y Duque sobre medio ambiente?"
resultados = coleccion.query(query_texts=[pregunta], n_results=4)

print(f"\nBúsqueda: {pregunta}")
print("Resultados:")

for i in range(len(resultados["documents"][0])):
    politico = resultados["metadatas"][0][i]['politico']
    similitud = 1 - resultados["distances"][0][i]
    texto = resultados["documents"][0][i][:100]

    print(f"{i+1}. {politico} ({similitud:.3f}) - {texto}...")

print("\n✅ RAG listo para usar!")


Búsqueda: ¿Qué diferencias hay entre Petro y Duque sobre medio ambiente?
Resultados:
1. Petro (-0.066) - Vengo de uno de los tres países más bellos de la Tierra.
Allí hay una explosión de vida. Miles de es...
2. Duque (-0.170) - Hoy desde esta majestuosa plaza que lleva el nombre de nuestro Libertador Simón Bolívar, quiero agra...
3. Duque (-0.193) - Nos encontramos de nuevo en este foro global que ha sido históricamente un espacio vital para el des...
4. Duque (-0.193) - Nos encontramos de nuevo en este foro global que ha sido históricamente un espacio vital para el des...

✅ RAG listo para usar!


## Preguntas

In [ ]:
# PREGUNTA 1: ¿Qué dice Petro sobre drogas y narcotráfico?

print("🔍 PREGUNTA 1: ¿Qué dice Petro sobre drogas y narcotráfico?")
print("="*60)

pregunta1 = "¿Qué dice Petro sobre drogas y narcotráfico?"

# Buscar documentos relevantes
resultados1 = coleccion.query(
    query_texts=[pregunta1],
    n_results=3
)


# Crear contexto
contexto1 = ""
for i in range(len(resultados1["documents"][0])):
    politico = resultados1["metadatas"][0][i]['politico']
    texto = resultados1["documents"][0][i]
    contexto1 += f"FUENTE {i+1} - {politico}: {texto}\n\n"

# Crear prompt
prompt1 = f"""
Basándote en estas fuentes, responde: {pregunta1}

{contexto1}

Responde de forma clara, citando específicamente qué dice Petro.
"""

# Generar respuesta
response1 = model.generate_content(prompt1)

print(f"\n💬 RESPUESTA:")
print(response1.text)
print("\n" + "="*60)

🔍 PREGUNTA 1: ¿Qué dice Petro sobre drogas y narcotráfico?

💬 RESPUESTA:
Basado en las fuentes proporcionadas, esto es lo que Petro dice sobre drogas y narcotráfico:

*   **Guerra contra las drogas ha fracasado:** "Claro que la paz es posible si se cambia, por ejemplo, la política contra las drogas, por ejemplo, vista como una guerra por una política de prevención fuerte del consumo en las sociedades desarrolladas." (Fuente 3) y "La guerra contra las drogas ha fracasado. La lucha contra la crisis climática ha fracasado." (Fuente 2).
*   **Nueva Convención Internacional:** "Es hora de una nueva Convención Internacional que acepte que la guerra contra las drogas ha fracasado..." (Fuente 3).
*   **Daño causado por la guerra contra las drogas:** Menciona que la guerra contra las drogas "ha dejado un millón de latinoamericanos asesinados, durante estos 40 años, y que deja 70.000 norteamericanos muertos por sobredosis cada año. Que la guerra contra las drogas fortaleció las mafias y debilitó

In [ ]:
# PREGUNTA 2: ¿Cuál es la posición de Duque sobre la pandemia?
# ===============================================

print("\n🔍 PREGUNTA 2: ¿Cuál es la posición de Duque sobre la pandemia?")
print("="*60)

pregunta2 = "¿Cuál es la posición de Duque sobre la pandemia?"

# Buscar documentos relevantes
resultados2 = coleccion.query(
    query_texts=[pregunta2],
    n_results=3
)

# Crear contexto
contexto2 = ""
for i in range(len(resultados2["documents"][0])):
    politico = resultados2["metadatas"][0][i]['politico']
    texto = resultados2["documents"][0][i]
    contexto2 += f"FUENTE {i+1} - {politico}: {texto}\n\n"

# Crear prompt
prompt2 = f"""
Basándote en estas fuentes, responde: {pregunta2}

{contexto2}

Responde de forma clara, citando específicamente qué dice Duque.
"""

# Generar respuesta
response2 = model.generate_content(prompt2)

print(f"\n💬 RESPUESTA:")
print(response2.text)
print("\n" + "="*60)


🔍 PREGUNTA 2: ¿Cuál es la posición de Duque sobre la pandemia?

💬 RESPUESTA:
Según las fuentes proporcionadas, la posición de Duque sobre la pandemia es la siguiente:

*   **Reconocimiento del impacto negativo:** Duque reconoce que la pandemia ha sido "cruel" y ha golpeado los sistemas de salud, las economías, la equidad y el avance de la Agenda 2030 (Fuente 1 y 2). Reconoce el dolor causado por la pérdida de seres queridos y los cambios en la vida cotidiana.
*   **Énfasis en la fragilidad humana y la capacidad de superación:** Destaca la "fragilidad del ser humano" frente a la pandemia, pero también reconoce "la grandeza y la condición de una raza humana que sabe sobreponerse a los grandes desafíos" (Fuente 1 y 2).
*   **Crítica al multilateralismo desigual:** Señala "fallas del multilateralismo para responder de manera equitativa y articulada a los momentos más agudos" y critica las "brechas existentes entre naciones respecto al proceso de vacunación" (Fuente 1 y 2). Menciona que al

In [ ]:
# PREGUNTA 3: ¿Qué diferencias hay entre sus enfoques económicos?
# ===============================================

print("\n🔍 PREGUNTA 3: ¿Qué diferencias hay entre sus enfoques económicos?")
print("="*60)

pregunta3 = "¿Qué diferencias hay entre Petro y Duque en sus enfoques económicos?"

# Buscar documentos relevantes
resultados3 = coleccion.query(
    query_texts=[pregunta3],
    n_results=4  # Más documentos para comparar
)


# Crear contexto
contexto3 = ""
for i in range(len(resultados3["documents"][0])):
    politico = resultados3["metadatas"][0][i]['politico']
    texto = resultados3["documents"][0][i]
    contexto3 += f"FUENTE {i+1} - {politico}: {texto}\n\n"

# Crear prompt
prompt3 = f"""
Basándote en estas fuentes, responde: {pregunta3}

{contexto3}

INSTRUCCIONES ESPECÍFICAS:
- Compara las posiciones de Petro vs Duque
- Identifica las principales diferencias
- Cita ejemplos específicos de cada uno
- Sé objetivo en la comparación

"""

# Generar respuesta
response3 = model.generate_content(prompt3)

print(f"\n💬 RESPUESTA:")
print(response3.text)
print("\n" + "="*60)


🔍 PREGUNTA 3: ¿Qué diferencias hay entre sus enfoques económicos?

💬 RESPUESTA:
Basado en las fuentes proporcionadas, estas son las principales diferencias en los enfoques económicos de Gustavo Petro y Iván Duque:

**1. Modelo Económico General:**

*   **Duque:** Se enfoca en un modelo de libre mercado con énfasis en el emprendimiento y la inversión privada. Busca reducir trámites innecesarios para facilitar la actividad empresarial y generar empleo. Quiere diversificar la producción y las exportaciones para no depender de recursos que se agoten.
    *   *Ejemplo:* Menciona la "Economía Naranja" y el apoyo a emprendedores tecnológicos como pilares del progreso.
*   **Petro:** Propone una economía basada en la producción, el trabajo y el conocimiento, con un fuerte énfasis en la redistribución de la riqueza para reducir la desigualdad. Busca una economía más productiva con el objetivo de generar una economía próspera pero legal que acabe con el atraso de las regiones. Para Petro la pro